## Update Requests

Este es el código de Web Scrapping para poder coseguir los resultados de los partidos de futbol y así posteriomente aplicar desde R la lógica necesaria para sumarle y quitarle puntos a los jugadores en función de las victorias y las derrotas que han tenido sus equipos.

### Librerías

Importamos la librerías necesarias

In [111]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

### Variables

Se definen todas las variables de nuestro entorno: la jornada que se quiere actualizar así como el URL al que se ahce la petición

In [112]:
jornada = "jornada10"
url = 'https://www.marca.com/futbol/primera-division/calendario.html?intcmp=MENUMIGA&s_kw=calendario'

Se hace la petición, lo parseamos a html para poder hacer las operaciones de búsqueda y filtrado y finalmente nos vamos a la sección de nuestra jornada en particular.

In [113]:
r = requests.get(url)
html = BeautifulSoup(r.content,"html.parser")
target_tables = html.find(id=jornada)

Definimos una función que nos sirve tanto para capturar los quipos locales como los visitantes, además del resultado obtenido

In [114]:
def getTotalCLubs(equipos, tipo):
    equiposBusqueda=target_tables.find_all(class_=tipo)
    for equipo in equiposBusqueda:
        equipos.append(equipo.find("span").text)
    return equipos

LLamamos a esa función para capturar los equipos

In [115]:
teams = []
teams=getTotalCLubs(teams, "local")
teams=getTotalCLubs(teams, "visitante")

Hacemos lo mismo para los resultados

In [116]:
res=[]
resultados = np.array(getTotalCLubs(res, "resultado"))

Nos definimos una función que capture cual de los dos equipos ha ganado haciendo las transformaciones necesarias

In [117]:
def ganador(x):
    if float(x.split("-")[0])>float(x.split("-")[1]):
        return 0.03
    elif float(x.split("-")[0])<float(x.split("-")[1]):
        return -0.03
    else:
        return 0

Se aplica la función a la primera mitas, ya que la segunda mitad es la inversa, es decir, si el local ha perdida implica que el visitante ha ganado.

In [118]:
res1 = np.array(list(map(ganador,resultados)))
res2 = res1*-1

Los unimos en un solo array

In [119]:
ponderacion = np.concatenate((res1,res2))

Generamos un DF

In [120]:
exportacion = pd.DataFrame({'Equpipo': team, 'Ponderacion': ponderacion})

In [121]:
exportacion

,Equpipo,Ponderacion
0,Osasuna,0.00
1,Levante,0.00
2,Villarreal,0.00
3,Sevilla,0.03
4,Atlético,0.03
5,Eibar,0.00
6,Cádiz,-0.03
7,Granada,-0.03
8,Alavés,0.00
9,Athletic,0.03


Y lo exportamos para poder trabajar con él en el futuro

In [122]:
exportacion.to_excel(jornada + '.xlsx')